# For data visulaization and EDA refer 'Residential_power_utilisation_data_visuals'

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
url = 'https://5vtj648dfk323byvjb7k1e9w-wpengine.netdna-ssl.com/wp-content/uploads/2018/05/shutterstock_170867918-e1525266245642.jpg'
Image(url= url, width=600, height=600, unconfined=True)

Image source from:- www.fleetcarma.com

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Import files
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt


from plotly.offline import iplot, init_notebook_mode, plot, download_plotlyjs
init_notebook_mode()

import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.subplots import make_subplots

# import sklearn 
from sklearn.linear_model import Ridge, LinearRegression, Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_selection import SelectKBest, mutual_info_regression

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score, max_error,median_absolute_error, mean_squared_log_error



In [ ]:
df_usage = pd.read_csv('../input/residential-power-usage-3years-data-timeseries/power_usage_2016_to_2020.csv')
df_weather = pd.read_csv('../input/residential-power-usage-3years-data-timeseries/weather_2016_2020_daily.csv')

In [ ]:
df_usage.head()

In [ ]:
df_weather.head()

# Date column update

In [ ]:
# Date column update for 'df_usage'

n = df_usage.shape[0]
p1 = pd.Series(range(n), pd.period_range('2016-06-01 00:00:00', freq = '1H', periods = n))
df_usage['StartDate'] = p1.to_frame().index

# Date column update for 'df_weather'
m = df_weather.shape[0]
p2 = pd.Series(range(m), pd.period_range('2016-06-01', freq = '1D', periods = m))
df_weather['Date'] = p2.to_frame().index

# convert the period date into timestamp
df_usage['StartDate'] = df_usage['StartDate'].apply (lambda x: x.to_timestamp())
df_usage['Date'] = pd.DatetimeIndex(df_usage['StartDate']).date

# convert the period date into timestamp
df_weather['Date'] = df_weather['Date'].apply (lambda x: x.to_timestamp())

# Group data by date

In [ ]:
df_usage_daily = df_usage.groupby('Date').sum()

df_usage_daily['day_of_week'] = df_usage_daily['day_of_week'].apply(lambda x: x/24)

notes_col = df_usage.groupby('Date').first()['notes'].values
df_usage_daily['notes'] = notes_col
df_usage_daily.head()

In [ ]:
#filter the weather data to match with power usage dataframe. 

k = df_usage_daily.shape[0]
df_weather[0:k]


#copy the value kwh into weather dataframe. 
df1 = df_usage_daily['Value (kWh)'].values
comb_df = df_weather[0:k]
comb_df['kWh_usage'] = pd.Series(df1).to_frame()


In [ ]:
comb_df['notes'] = notes_col
corr_df = comb_df[['Temp_max', 'Temp_min', 'Dew_max','Dew_min', 'kWh_usage', 'notes']]

power_split = df_usage['Value (kWh)'].values

data_new = power_split
split_data_power = (list([data_new[x:x+24] for x in range(0, len(data_new), 24)]))
#print(len(split_data_power)) 

#comb_df['power_array'] = pd.Series(split_data).to_frame()
comb_df['power_array_nsaled'] = pd.Series(split_data_power).to_frame()
comb_df.head()

In [ ]:
g = sns.PairGrid(corr_df, hue="notes")
g.map_diag(plt.hist)
g.map_offdiag(plt.scatter)
g.add_legend();

In [ ]:
sns.heatmap(corr_df.corr(), annot= True )

In [ ]:
# function to evaulate performance of the regressor. 

def evaulation(model, y_pred, y_true):
    
    '''
    Input:- model = string (Name of the regressor)
    y_pred= model prediction
    y_true = actual labels. 
    
    Output:
    Dataframe with evaulation matrix. 
    
    '''
    
    # create data output frame for the evaluation. 
    data = [explained_variance_score(y_true,y_pred), 
            max_error(y_true,y_pred),
            mean_squared_error(y_true,y_pred),
            mean_absolute_error(y_true,y_pred),
            r2_score(y_true,y_pred, multioutput='uniform_average'),
            median_absolute_error(y_true,y_pred)           
            ]
    row_index = ['Exp_Var_Score', 'Max_Error','MSE','MAE','R2_Score', 'Median_Abs_Error']
    
    df = pd.DataFrame(data, columns= [model], index= row_index)
    
    return df

In [ ]:
X_new = comb_df[['Temp_max', 'Temp_min', 'Dew_max', 'Precipit']]
y_new = comb_df['kWh_usage'].values

In [ ]:
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_new, y_new, test_size = 0.3, random_state = 0)

# Linear Regression

In [ ]:
# Step2 Initiate linear regressor
lr = LinearRegression()

# step3 fit the data
lr.fit(X_train_new, y_train_new)

# predict the test data
y_pred_lr_new = lr.predict(X_test_new)


#evaulation of the lr   

print('Intercept:', lr.intercept_)
print('Coefficients:', lr.coef_)


In [ ]:
# Linear regressor evaulation parameters
df_linear = evaulation('linear', y_pred_lr_new, y_test_new)
df_linear


# SVR non linear

In [ ]:

svr = SVR(C=11, epsilon=1, kernel='rbf', gamma=3, tol = .001, verbose = 0)


pipe = Pipeline( steps = [('Standardscaler', StandardScaler()), 
                          ('SVR', svr)])

pipe.fit(X_train_new, y_train_new)

y_pred_svr = pipe.predict(X_test_new)

In [ ]:
df_svr = evaulation('SVR', y_pred_svr, y_test_new)
df_svr

In [ ]:
#sample check
X_test_new[:10]

In [ ]:
#Sample check 
pipe.predict(X_test_new[:10].values)

In [ ]:
y_test_new[:10]

In [ ]:
#from the above prediction the error for item 1 and 6 are high in comparision to other

# Conclusions

1. The R2 square value for regressor is less than .7, the predictions will have errors. 
2. Since the number of readings are less than 1500 and non linear. The prediction shall have high errors
3. Performed classification analysis to predict "weekend, weekday, vacation and covid" power consumption patterns. However the efficieny of prediction was very low. (The mean consumption for weekend, weekday and covid are not statisitical different)